# Installation , Import Libraries and Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/my_drive', force_remount=True)

# Navigate to the project directory
%cd '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/'

# Define dataset path
dataset_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset'




Mounted at /content/my_drive
/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection


In [2]:
!pip install ultralytics torch torchvision torchaudio mlflow pyngrok pyyaml --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 923.8/923.8 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# General libraries
import os
import time
import numpy as np
#import mlflow
from PIL import Image

from ultralytics import YOLO
import ultralytics

import datetime

import subprocess

# MLflow
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow.pyfunc


# Google Colab tools
from google.colab import drive

# For ngrok to expose the MLflow UI remotely
#from pyngrok import ngrok

# Visualization
import matplotlib.pyplot as plt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Train YOLOv11n

- This part starts the experiment, logs parameters, and trains the YOLOv11n model.

In [8]:
def train_yolo(model_path, dataset_path, log_path, exp_id=datetime.datetime.now().strftime('%Y%m%d%H%M%S'), epochs=50, lr=0.001, optimizer='AdamW', resume=False):


    with mlflow.start_run(run_name=exp_id):
        # Log hyperparameters
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("img_size", 640)
        mlflow.log_param("batch_size", 64)

        # Start timer for training duration
        start_time = time.time()

        if resume:
            # Load a model
            model = YOLO(model_path)  # load a partially trained model
            # Resume training
            results = model.train(resume=True)
        else:
            # Hyperparameters for training YOLOv11n
            params = {
                'exp_id': exp_id,
                'epochs': epochs,
                'img_size': 640,
                'batch_size': 64,
                'lr': lr,
                'optimizer': optimizer,
                'save_period': 10,
                'single_cls': True
            }


            # Load the YOLOv11n model
            model = YOLO(model_path)  # Load YOLOv11n or YOLOv11s model with custom weights

            # Train the model with updated parameters
            results = model.train(
                data = dataset_path,
                epochs = params['epochs'],
                imgsz = params['img_size'],
                batch = params['batch_size'],
                lr0 = params['lr'],
                optimizer = params['optimizer'],
                project = log_path,
                name = params['exp_id'],
                save_period=params['save_period'],
                single_cls=params['single_cls']
            )


        # Log training time
        training_duration = time.time() - start_time
        mlflow.log_metric("training_duration", training_duration)
        print(f"Training completed in {training_duration:.2f} seconds")

        return results


### Additional Augmentation Parameters

In [ ]:
"""def train_yolo(model_path, dataset_path, log_path, exp_id=datetime.datetime.now().strftime('%Y%m%d%H%M%S'),
               epochs=50, lr=0.001, optimizer='AdamW', resume=False):




    with mlflow.start_run(run_name=exp_id):
        # Log hyperparameters
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("learning_rate", lr)
        mlflow.log_param("optimizer", optimizer)
        mlflow.log_param("img_size", 640)
        mlflow.log_param("batch_size", 64)

        # Start timer for training duration
        start_time = time.time()

        if resume:
            # Load a model
            model = YOLO(model_path)  # load a partially trained model
            # Resume training
            results = model.train(resume=True)
        else:
            # Hyperparameters for training YOLOv11n
            params = {
                'exp_id': exp_id,
                'epochs': epochs,
                'img_size': 640,
                'batch_size': 64,
                'lr': lr,
                'optimizer': optimizer,
                'save_period': 10
            }

            # Load the YOLOv11n model
            model = YOLO(model_path)  # Load YOLOv11n model with custom weights

            # Train the model with updated parameters and heavy augmentation overrides
            results = model.train(
                data=dataset_path,
                epochs=params['epochs'],
                imgsz=params['img_size'],
                batch=params['batch_size'],
                lr0=params['lr'],
                optimizer=params['optimizer'],
                project=log_path,
                name=params['exp_id'],
                save_period=params['save_period'],
                augment=True,
                hsv_h=0.01,       # Subtle hue shift (minimal change, important if images are near grayscale)
                hsv_s=0.4,        # Moderate saturation adjustment
                hsv_v=0.3,        # Moderate brightness/value variation
                degrees=10.0,     # Small rotation (within 5-15°) to simulate slight misalignment
                translate=0.1,    # Minor translation (about 10% shift) for patient positioning variability
                scale=0.3,        # Moderate scaling variation (up to 30% size change)
                shear=0.0,        # No shearing; keeps the bone structure intact
                perspective=0.0,  # No perspective transform
                mixup=0.1         # Low mixup probability to provide some regularization without excessive blending
            )

        # Log training time
        training_duration = time.time() - start_time
        mlflow.log_metric("training_duration", training_duration)
        print(f"Training completed in {training_duration:.2f} seconds")

        return results


In [9]:
# Set MLflow tracking
mlflow_tracking_dir = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow'
mlflow.set_tracking_uri(f"file://{mlflow_tracking_dir}")


mlflow.set_experiment("Base_Bone_Fraction_Detection")


<Experiment: artifact_location='file:///content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow/669269712119322704', creation_time=1735159927933, experiment_id='669269712119322704', last_update_time=1735159927933, lifecycle_stage='active', name='Base_Bone_Fraction_Detection', tags={}>

In [10]:
mlflow.end_run()

# Dataset path (ensure this is where your data is located)
dataset_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/data.yaml'

# Pre-trained model path
model_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/models/yolo11s.pt'   # change the model from n to s
#model_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/04/weights/epoch50.pt'   # consider as my pretrined model, 04 should change to Exp2

# Log path
log_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints'

# Parameters
exp_id='07'
epochs= 60
lr=0.001


results = train_yolo(model_path,
                     dataset_path,
                     log_path,
                     exp_id=exp_id,
                     epochs=epochs,
                     lr=lr,
                     resume=False
                     )


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/models/yolo11s.pt, data=/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/data.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=8, project=/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints, name=07, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_

100%|██████████| 755k/755k [00:00<00:00, 70.9MB/s]

Overriding class names with single class.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/train/labels... 3779 images, 2909 backgrounds, 0 corrupt: 100%|██████████| 3779/3779 [03:13<00:00, 19.49it/s]


...

      60/60      14.3G     0.7151     0.4749      1.056          0        640: 100%|██████████| 60/60 [00:59<00:00,  1.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.11it/s]

                   all        835       1171      0.781      0.546      0.625      0.247



60 epochs completed in 1.124 hours.
Optimizer stripped from /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/07/weights/last.pt, 19.2MB
Optimizer stripped from /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/07/weights/best.pt, 19.2MB

Validating /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/07/weights/best.pt...
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:06<00:00,  1.00it/s]


                   all        835       1171      0.782      0.545      0.625      0.247
Speed: 0.2ms preprocess, 4.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/07
MLflow: results logged to file:///content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'
Training completed in 4311.68 seconds


# Test

In [12]:
dataset_config_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/data.yaml'

# Load your trained YOLO11 model (update with your model path)
model = YOLO("/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/07/weights/best.pt")

# Run validation (testing) on the dataset defined in data.yaml.
# You can adjust parameters such as image size (imgsz), confidence threshold (conf), and IoU threshold (iou).
metrics = model.val(data = dataset_config_path, split="test")


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/test/labels.cache... 841 images, 441 backgrounds, 0 corrupt: 100%|██████████| 841/841 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 53/53 [00:11<00:00,  4.44it/s]


                   all        841        783      0.755      0.599      0.673      0.264
Speed: 0.4ms preprocess, 8.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/val6


In [ ]:
# Run MLflow UI in the background
import subprocess
from pyngrok import ngrok

subprocess.Popen(["mlflow", "ui", "--backend-store-uri", mlflow_tracking_dir])

# Set up Ngrok for remote access
NGROK_AUTH_TOKEN = "2qGXY4i9cF4CQHjBRFLtxjuQCTz_Z9PruXmBce21Hfpju7AY"  # Replace with your Ngrok token
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for accessing MLflow UI
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)

print("MLflow Tracking UI:", ngrok_tunnel.public_url)

# Terminate open tunnels if exist
# ngrok.kill()


In [ ]:
# nference on a specific image from your training dataset using a trained YOLO model
image_list = os.listdir('/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/train/images')

# Run inference on a test image
test_image_path = os.path.join("/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Fraction_Detection_Dataset/train/images/", image_list[0])  # Example image index

# Load the YOLOv11n model (ensure the weights file exists at the given path)
model_inf_path = '/content/my_drive/MyDrive/Project_Experiments/Bone_Fraction_Detection/Model_Checkpoints/032/weights/last.pt'
model_inf = YOLO(model_inf_path)  # Load YOLOv11n model with custom weights
results = model_inf(test_image_path)  # Get predictions for the test image

# Visualize the results
for r in results:
    # Plot the prediction results
    im_bgr = r.plot()  # Convert to BGR
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # Convert to RGB
    r.show()  # Show the result
